# 02: Running the Agent

In Notebook 01 we explored the dataset and tools. This notebook shows how to run the
`KnowledgeGroundedAgent` in practice.

## What You'll Learn

1. The agent's PlanReAct architecture and the `AgentResponse` data structure
2. Running a question with live progress display
3. Inspecting the response: plan, tool calls, sources, and reasoning
4. Multi-turn conversations using session state
5. Observability with Langfuse tracing

## Prerequisites

Complete Notebook 01. You'll need `GOOGLE_API_KEY` in your `.env` file.
For tracing (Section 4): `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_SECRET_KEY` are also required.

In [1]:
import os
import uuid
from pathlib import Path

from aieng.agent_evals.knowledge_qa import KnowledgeGroundedAgent
from aieng.agent_evals.knowledge_qa.notebook import display_response, run_with_display
from aieng.agent_evals.langfuse import init_tracing
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel
from rich.table import Table


if Path("").absolute().name == "eval-agents":
    print(f"Working directory: {Path('').absolute()}")
else:
    os.chdir(Path("").absolute().parent.parent)
    print(f"Working directory set to: {Path('').absolute()}")

load_dotenv(verbose=True)
console = Console(width=100)

Working directory set to: /Users/amritkrishnan/src/eval-agents


## 1. Agent Architecture

The `KnowledgeGroundedAgent` is built on Google ADK and combines two patterns:

**PlanReAct** — Before executing, the agent produces an explicit research plan with numbered
steps. Each step has a type (`SEARCH`, `FETCH`, `ANALYZE`) and a status that transitions from
`pending` → `in_progress` → `completed` (or `failed`/`skipped`). The plan can be revised
mid-run if the agent encounters unexpected results.

**ReAct loop** — Within each step, the agent alternates between reasoning (Thought), acting
(tool call), and observing (tool response).

### The `AgentResponse` Object

After running, `agent.answer_async(question)` returns an `AgentResponse`:

| Field | Type | Description |
|-------|------|-------------|
| `text` | `str` | The final answer |
| `plan` | `ResearchPlan` | Numbered steps with statuses |
| `tool_calls` | `list[dict]` | Every tool invocation during execution |
| `sources` | `list[GroundingChunk]` | URLs used as evidence |
| `reasoning_chain` | `list[str]` | The model's intermediate reasoning |
| `total_duration_ms` | `int` | Wall-clock execution time |

In [2]:
agent = KnowledgeGroundedAgent(enable_planning=True)

config_table = Table(title="Agent Configuration", show_header=False)
config_table.add_column("Setting", style="cyan")
config_table.add_column("Value", style="white")
config_table.add_row("Model", agent.model)
config_table.add_row("Planning", "PlanReAct (enabled)")
config_table.add_row("Session Service", "InMemorySessionService")
config_table.add_row("Tools", "google_search, web_fetch, fetch_file, grep_file, read_file")

console.print(config_table)

                              Agent Configuration                               
┌─────────────────┬────────────────────────────────────────────────────────────┐
│ Model           │ gemini-2.5-flash                                           │
│ Planning        │ PlanReAct (enabled)                                        │
│ Session Service │ InMemorySessionService                                     │
│ Tools           │ google_search, web_fetch, fetch_file, grep_file, read_file │
└─────────────────┴────────────────────────────────────────────────────────────┘

## 2. Running a Question

`run_with_display` executes the agent in a Jupyter notebook with a live progress display showing:

- The research plan with step statuses (updating in real time)
- Tool calls as they fire

We'll use a question that requires web search — the agent must find and verify a specific fact,
not recall it from training data.

In [3]:
question = "When was the highest single-day snowfall recorded in Toronto, and how much snow fell?"

response = await run_with_display(agent, question)

In [4]:
display_response(
    console,
    response.text,
    subtitle=(
        f"Duration: {response.total_duration_ms / 1000:.1f}s  |  "
        f"Tool calls: {len(response.tool_calls)}  |  "
        f"Sources: {len(response.sources)}"
    ),
)

╭───────────────────────────────────────────── Answer ─────────────────────────────────────────────╮
│ The highest single-day snowfall recorded in Toronto was 65 cm, which occurred on January 25,     │
│ 2026, in Scarborough (South).                                                                    │
╰─────────────────────── Duration: 71.6s  |  Tool calls: 9  |  Sources: 23 ────────────────────────╯

╭─────────────────────────────────────────── Reasoning ────────────────────────────────────────────╮
│ According to citynews.ca, on January 25, 2026, Scarborough (South) received 65 cm of snow, and   │
│ downtown Toronto received 61 cm. CP24.com also reported an estimated 56 cm in downtown Toronto   │
│ on the same date. The 65 cm in Scarborough (South) is the highest amount reported for a single   │
│ day in Toronto.                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### 2.1 Inspecting the Response

The `AgentResponse` object contains the full execution trace.

In [5]:
plan = response.plan

plan_table = Table(title="Research Plan")
plan_table.add_column("#", style="cyan", width=3)
plan_table.add_column("Step", style="white")
plan_table.add_column("Type", style="dim", width=12)
plan_table.add_column("Status", style="green")

for step in plan.steps:
    icon = {"completed": "✓", "failed": "✗", "skipped": "○"}.get(step.status.value, "·")
    desc = step.description[:70] + "..." if len(step.description) > 70 else step.description
    plan_table.add_row(str(step.step_id), desc, step.step_type, f"{icon} {step.status.value}")

console.print(plan_table)

                                           Research Plan                                            
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ #   ┃ Step                                                          ┃ Type         ┃ Status      ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 1   │ Search for "highest single-day snowfall in Toronto" to find   │ research     │ ✓ completed │
│     │ relevant a...                                                 │              │             │
│ 2   │ From the search results, identify a reliable source (e.g.,    │ research     │ ✓ completed │
│     │ official we...                                                │              │             │
│ 3   │ Use `web_fetch` to extract the specific date and snowfall     │ research     │ ✓ completed │
│     │ amount from ...                                               │              │             │
│ 4   │ Verify the information and provide the final answer.          │ research     │ ✓ completed │
└─────┴───────────────────────────────────────────────────────────────┴──────────────┴─────────────┘

In [6]:
if response.tool_calls:
    tools_table = Table(title="Tool Calls")
    tools_table.add_column("#", style="dim", width=3)
    tools_table.add_column("Tool", style="cyan", width=16)
    tools_table.add_column("Arguments (truncated)", style="white")

    for i, tc in enumerate(response.tool_calls[:15], 1):
        name = tc.get("name", "unknown")
        args = str(tc.get("args", {}))
        args = args[:70] + "..." if len(args) > 70 else args
        tools_table.add_row(str(i), name, args)

    if len(response.tool_calls) > 15:
        tools_table.add_row("...", f"({len(response.tool_calls) - 15} more)", "")

    console.print(tools_table)
else:
    console.print("[dim]No tool calls recorded[/dim]")

                                             Tool Calls                                             
┏━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ #   ┃ Tool             ┃ Arguments (truncated)                                                   ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ google_search    │ {'query': 'highest single-day snowfall in Toronto'}                     │
│ 2   │ web_fetch        │ {'url':                                                                 │
│     │                  │ 'https://torontolife.com/city/yesterdays-snowfall-was-the-bigg...       │
│ 3   │ web_fetch        │ {'query': 'date and amount of highest single-day snowfall in            │
│     │                  │ Toronto',...                                                            │
│ 4   │ web_fetch        │ {'url':                                                                 │
│     │                  │ 'https://toronto.citynews.ca/2026/01/26/toronto-pearson-snowfa...       │
│ 5   │ google_search    │ {'query': 'highest single-day snowfall in Toronto'}                     │
│ 6   │ web_fetch        │ {'query': 'highest single-day snowfall in Toronto and amount', 'url':   │
│     │                  │ ...                                                                     │
│ 7   │ web_fetch        │ {'url':                                                                 │
│     │                  │ 'https://www.theguardian.com/world/2026/jan/26/toronto-snowsto...       │
│ 8   │ web_fetch        │ {'query': 'highest single-day snowfall in Toronto and amount', 'url':   │
│     │                  │ ...                                                                     │
│ 9   │ google_search    │ {'query': 'all-time highest single-day snowfall Toronto record'}        │
└─────┴──────────────────┴─────────────────────────────────────────────────────────────────────────┘

In [7]:
if response.sources:
    seen: set[str] = set()
    sources_table = Table(title="Sources")
    sources_table.add_column("#", style="dim", width=3)
    sources_table.add_column("URL", style="blue")

    for src in response.sources:
        if src.uri and src.uri not in seen:
            seen.add(src.uri)
            url = src.uri[:85] + "..." if len(src.uri) > 85 else src.uri
            sources_table.add_row(str(len(seen)), url)
        if len(seen) >= 10:
            break

    console.print(sources_table)
else:
    console.print("[dim]No sources recorded[/dim]")

                                             Sources                                              
┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ #   ┃ URL                                                                                      ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ https://www.currentresults.com/Yearly-Weather/Canada/ON/Toronto/extreme-annual-toront... │
│ 2   │ https://www.torontotoday.ca/local/environment-climate/toronto-breaks-snowfall-records... │
│ 3   │ https://torontolife.com/city/yesterdays-snowfall-was-the-biggest-single-day-snowstorm... │
│ 4   │ https://www.theweathernetwork.com/en/news/weather/severe/photos-major-winter-storm-hi... │
│ 5   │ https://toronto.citynews.ca/2026/01/26/toronto-pearson-snowfall-record-sunday-january... │
│ 6   │ https://snowbrains.com/toronto-just-faced-its-snowiest-day-on-record/                    │
│ 7   │ https://www.cbc.ca/news/canada/toronto-snow-military-help-9.7061297                      │
│ 8   │ https://www.cp24.com/local/toronto/2026/01/26/what-a-winter-wallop-storm-sets-new-sin... │
│ 9   │ https://toronto.citynews.ca/2008/02/08/what-is-the-all-time-single-day-winter-snow-re... │
│ 10  │ https://www.ctvnews.ca/toronto/article/what-a-winter-wallop-storm-sets-new-single-day... │
└─────┴──────────────────────────────────────────────────────────────────────────────────────────┘

## 3. Multi-Turn Conversations

The agent uses an `InMemorySessionService` to maintain conversation context across turns.
Pass the same `session_id` to link questions together — the agent will use prior context
when answering follow-up questions.

In [8]:
session_id = str(uuid.uuid4())
console.print(f"Session ID: [dim]{session_id}[/dim]\n")

# First turn: establish a subject
response1 = await agent.answer_async(
    "What is the capital of France?",
    session_id=session_id,
)
display_response(console, response1.text, title="Turn 1")

# Second turn: follow-up that references the prior context
response2 = await agent.answer_async(
    "What is the official language spoken there?",
    session_id=session_id,
)
display_response(console, response2.text, title="Turn 2 (follow-up)")

Session ID: 49f5f858-8b9d-40eb-83eb-002c2916c5a1

2026-02-25 01:13:11,783 INFO aieng.agent_evals.knowledge_qa.agent: Answering question: What is the capital of France?...
2026-02-25 01:13:11,798 INFO google_adk.google.adk.models.google_llm: Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2026-02-25 01:13:13,076 INFO google_adk.google.adk.models.google_llm: Response received from the model.
2026-02-25 01:13:13,078 INFO aieng.agent_evals.knowledge_qa.agent: Extracted plan with 4 steps
2026-02-25 01:13:13,079 INFO aieng.agent_evals.knowledge_qa.agent: Tool call: google_search({'query': 'capital of France'})
2026-02-25 01:13:13,102 INFO google_genai.models: AFC is enabled with max remote calls: 10.
2026-02-25 01:13:19,063 WARNING aieng.agent_evals.tools._redirect: Failed to resolve redirect URL https://vertexaisearch.cloud.google.com/grounding-api-redire...: RetryError: RetryError[<Future at 0x11ca6a2a0 state=finished raised ConnectError>]
2026-02-25 01:13:19,069 INFO aieng.agent_evals.kno

╭───────────────────────────────────────────── Turn 1 ─────────────────────────────────────────────╮
│ The capital of France is Paris.                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Sources ─────────────────────────────────────────────╮
│   https://www.britannica.com/place/France                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Reasoning ────────────────────────────────────────────╮
│ The Britannica page for France explicitly states "Capital: : Paris" in its Quick Facts section   │
│ and also in the main body text, "The capital and by far the most important city of France is     │
│ Paris".                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

2026-02-25 01:13:27,600 INFO aieng.agent_evals.knowledge_qa.agent: Answering question: What is the official language spoken there?...
2026-02-25 01:13:27,620 INFO google_adk.google.adk.models.google_llm: Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2026-02-25 01:13:27,620 INFO google_genai.models: AFC is enabled with max remote calls: 10.
2026-02-25 01:13:28,849 INFO google_adk.google.adk.models.google_llm: Response received from the model.
2026-02-25 01:13:28,851 INFO aieng.agent_evals.knowledge_qa.agent: Tool call: google_search({'query': 'official language of France'})
2026-02-25 01:13:28,875 INFO google_genai.models: AFC is enabled with max remote calls: 10.
2026-02-25 01:13:32,085 INFO aieng.agent_evals.knowledge_qa.agent: Tool response: google_search completed
2026-02-25 01:13:32,111 INFO google_adk.google.adk.models.google_llm: Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
20

╭─────────────────────────────────────── Turn 2 (follow-up) ───────────────────────────────────────╮
│ The official language spoken in France is French.                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Sources ─────────────────────────────────────────────╮
│   https://www.polilingua.com/blog/post/languages-of-france-and-french-dialects.htm               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Reasoning ────────────────────────────────────────────╮
│ The Polilingua blog post states, "The official language of France is French." This verifies the  │
│ information from the initial Google search summary.                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

## 4. Observability with Langfuse

Langfuse captures a full trace of every agent run using OpenTelemetry, giving you visibility into:

- Every tool call and its arguments
- Every LLM call with prompts and completions
- Timing for each span
- The full agent execution tree

This is essential for debugging failures, measuring latency, and comparing configurations.

### Trace Structure

```
Trace: agent run
├── Span: planning (PlanReAct)
│   └── LLM Call: create_plan
├── Span: step-1-execution
│   ├── Tool Call: google_search
│   ├── Tool Call: web_fetch
│   └── LLM Call: step_summary
├── Span: step-2-execution
│   └── ...
└── Span: synthesis
    └── LLM Call: final_answer
```

### Prerequisites

Set these in your `.env` file:
- `LANGFUSE_PUBLIC_KEY`
- `LANGFUSE_SECRET_KEY`
- `LANGFUSE_HOST` (optional, defaults to `https://cloud.langfuse.com`)

In [9]:
langfuse_configured = all(
    [
        os.getenv("LANGFUSE_PUBLIC_KEY"),
        os.getenv("LANGFUSE_SECRET_KEY"),
    ]
)

if langfuse_configured:
    console.print("[green]✓[/green] Langfuse credentials found")
else:
    console.print("[yellow]⚠[/yellow] Langfuse credentials not found — tracing cells will be skipped")
    console.print("[dim]Set LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY in .env[/dim]")

✓ Langfuse credentials found

In [10]:
tracing_enabled = init_tracing()

if tracing_enabled:
    console.print("[green]✓[/green] Langfuse tracing initialized")
else:
    console.print("[yellow]⚠[/yellow] Tracing not enabled (check credentials)")

2026-02-25 01:13:41,689 WARNING opentelemetry.trace: Overriding of current TracerProvider is not allowed
2026-02-25 01:13:41,848 INFO aieng.agent_evals.langfuse: Langfuse tracing initialized successfully (endpoint: https://us.cloud.langfuse.com/api/public/otel)


✓ Langfuse tracing initialized

In [11]:
if tracing_enabled:
    from langfuse import Langfuse

    langfuse = Langfuse()
    traced_agent = KnowledgeGroundedAgent(enable_planning=True)
    traced_question = "What programming language was created by Guido van Rossum, and in what year?"

    console.print(Panel(traced_question, title="Traced Question", border_style="green"))

    with langfuse.start_as_current_span(name="knowledge-agent", input=traced_question):
        trace_id = langfuse.get_current_trace_id()
        traced_response = await traced_agent.answer_async(traced_question)
        langfuse.update_current_span(output=traced_response.text)

    display_response(
        console,
        traced_response.text,
        subtitle=f"Duration: {traced_response.total_duration_ms / 1000:.1f}s",
    )
else:
    console.print("[dim]Skipping (Langfuse not configured)[/dim]")
    trace_id = None

╭──────────────────────────────────────── Traced Question ─────────────────────────────────────────╮
│ What programming language was created by Guido van Rossum, and in what year?                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

2026-02-25 01:13:41,956 INFO aieng.agent_evals.knowledge_qa.agent: Answering question: What programming language was created by Guido van Rossum, and in what year?...
/Users/amritkrishnan/src/eval-agents/.venv/lib/python3.12/site-packages/google/adk/models/google_llm.py:172: UserWarning: [EXPERIMENTAL] GeminiContextCacheManager: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  cache_manager = GeminiContextCacheManager(self.api_client)
2026-02-25 01:13:41,969 INFO google_adk.google.adk.models.google_llm: Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2026-02-25 01:13:43,492 INFO google_adk.google.adk.models.google_llm: Response received from the model.
2026-02-25 01:13:43,498 INFO aieng.agent_evals.knowledge_qa.agent: Extracted plan with 2 steps
2026-02-25 01:13:43,499 INFO aieng.agent_evals.knowledge_qa.agent: Tool call: google_search({'query':

╭───────────────────────────────────────────── Answer ─────────────────────────────────────────────╮
│ The programming language created by Guido van Rossum is Python. He began implementing it in      │
│ December 1989.                                                                                   │
╰──────────────────────────────────────── Duration: 18.8s ─────────────────────────────────────────╯

╭──────────────────────────────────────────── Sources ─────────────────────────────────────────────╮
│   https://www.wajusoft.com/blog/when-was-python-developed-all-your-questions-answered            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Reasoning ────────────────────────────────────────────╮
│ The article from wajusoft.com states: "Python was created by Guido van Rossum, a Dutch           │
│ programmer. It was conceived in December 1989."                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
if tracing_enabled:
    from IPython.display import HTML, display  # noqa: A004
    from langfuse import Langfuse
    from opentelemetry import trace as otel_trace

    provider = otel_trace.get_tracer_provider()
    if hasattr(provider, "force_flush"):
        provider.force_flush(timeout_millis=5000)
    console.print("[green]✓[/green] Traces flushed to Langfuse")

    if trace_id:
        trace_url = Langfuse().get_trace_url(trace_id=trace_id)
        display(HTML(f'<p>View trace: <a href="{trace_url}" target="_blank">{trace_url}</a></p>'))

✓ Traces flushed to Langfuse

### 4.1 Viewing Traces in the Langfuse UI

Open your Langfuse project and navigate to **Traces**. Each run appears as a
tree of spans. Useful things to look at:

- **Span timeline** — which steps take the most time?
- **Tool call arguments** — what search queries did the agent use?
- **LLM interactions** — what did the model reason about before calling each tool?
- **Errors** — red spans show where failures occurred

You can also filter by trace name, time range, or input content.

## Summary

In this notebook you learned:

1. **Creating the agent** — `KnowledgeGroundedAgent(enable_planning=True)` with PlanReAct
2. **Running questions** — `run_with_display` for live notebook progress; `agent.answer_async` for raw access
3. **The `AgentResponse`** — plan, tool calls, sources, reasoning, and timing in one object
4. **Multi-turn conversations** — linking turns with `session_id`
5. **Langfuse tracing** — `init_tracing()` and the Langfuse SDK for full observability

**Next:** In Notebook 03, we'll run a systematic evaluation using the DeepSearchQA benchmark.

In [13]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]03_evaluation.ipynb[/bold] to evaluate the agent at scale.",
        title="Done",
        border_style="green",
    )
)

╭────────────────────────────────────────────── Done ──────────────────────────────────────────────╮
│ ✓ Notebook complete!                                                                             │
│                                                                                                  │
│ Next: Open 03_evaluation.ipynb to evaluate the agent at scale.                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯